In [1]:
import numpy
import sys

In [2]:
def SET_PROPENSITIES(observed_ratings, inverse_propensities, verbose = False):
    numObservations = numpy.ma.count(observed_ratings)
    numUsers, numItems = numpy.shape(observed_ratings)
    scale = numUsers * numItems
    inversePropensities = None
    if inverse_propensities is None: # 初始化inverse_propensities
        inversePropensities = numpy.ones((numUsers, numItems), dtype = numpy.longdouble) * scale /
                            numObservations
    else: # 将inverse_propensities存入array
        inversePropensities = numpy.array(inverse_propensities, dtype = numpy.longdouble, copy = True)
    
    # 提取inversePropensities中仅包含observed的子集
    inversePropensities = numpy.ma.array(inversePropensities, dtype = numpy.longdouble, copy = False, 
                            mask = numpy.ma.getmask(observed_ratings), fill_value = 0, hard_mask = True)
 
    if verbose: #输出结果
        print ("Metrics.SET_PROPENSITIES: [LOG]\t NumUsers, NumItems, NumObservations",
            numUsers, numItems, numObservations)
        print ("Metrics.SET_PROPENSITIES: [DBG]\t Sum of observed inverse propensities ",
            numpy.ma.sum(inversePropensities, dtype = numpy.longdouble),
            ("(=? NumUsers * NumItems)"), numUsers * numItems)

    return inversePropensities

SyntaxError: invalid syntax (Temp/ipykernel_13332/1305855443.py, line 7)

In [1]:
import pandas as pd
import numpy as np

In [2]:
rnames = ['userid','itemid','rating','timestamp']
rating = pd.read_table(r'ml-100k/u.data',names = rnames,engine = 'python')
rating = rating.set_index(['userid','itemid'],drop=False) 
rating.head()

,,userid,itemid,rating,timestamp
userid,itemid,,,,
196,242,196,242,3,881250949
186,302,186,302,3,891717742
22,377,22,377,1,878887116
244,51,244,51,2,880606923
166,346,166,346,1,886397596


In [3]:
rating_pair = rating.drop(['rating', 'timestamp'], axis = 1)

In [4]:
unames = ['userid','age','gender','occupation','zip code']
users = pd.read_table(r'ml-100k/u.user',sep = '\|',names = unames, engine = 'python')
users.head()

,userid,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [5]:
inames = ['itemid','movie title','release date','video release date','IMDB URL','unknown','Action','Adventure'
          ,'Animation','Children\'s','Comedy','Crime','Documentrary','Drama','Fatasy','Film-Noir','Horror','Musical'
          ,'Mystery','Romance','Sci-Fi','Thriller','War','Western']
items = pd.read_table(r'ml-100k/u.item',sep = '\|',names = inames,engine = 'python',encoding='latin-1')
items.head()

,itemid,movie title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,...,Fatasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
items_matrix = items[['itemid', 'unknown', 'Action', 'Adventure', 'Animation',
                     'Children\'s', 'Comedy', 'Crime', 'Documentrary', 'Drama',
                     'Fatasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                     'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']]
items_matrix.head()

,itemid,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentrary,Drama,Fatasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [7]:
num_items = len(items)
num_users = len(users)
num_observation = np.ma.count(rating_pair)
scale = num_items * num_users
print(num_items, num_users, scale, num_observation)

1682 943 1586126 200000


In [8]:
def calMovies(userid):
    #观影记录
    try:
        views = rating.loc[(userid,slice(None)),:]
    except:
        return []
    movies = views['itemid'].values.tolist()
    return movies
users['movies'] = users['userid'].apply(calMovies)
users = users[['userid', 'movies']]
users

,userid,movies
0,1,"[61, 189, 33, 160, 20, 202, 171, 265, 155, 117..."
1,2,"[292, 251, 50, 314, 297, 290, 312, 281, 13, 28..."
2,3,"[335, 245, 337, 343, 323, 331, 294, 332, 328, ..."
3,4,"[264, 303, 361, 357, 260, 356, 294, 288, 50, 3..."
4,5,"[2, 17, 439, 225, 110, 454, 424, 1, 363, 98, 1..."
...,...,...
938,939,"[931, 106, 258, 1054, 689, 476, 409, 121, 1190..."
939,940,"[193, 568, 14, 205, 272, 655, 315, 66, 873, 28..."
940,941,"[147, 124, 117, 181, 993, 258, 7, 475, 257, 15..."
941,942,"[117, 200, 604, 423, 261, 427, 487, 323, 615, ..."


In [18]:
observed_matrix = users.drop('movies', 1).join(
    pd.get_dummies(pd.DataFrame(
        users.movies.tolist()).stack()).astype(int).sum(level=0))
observed_matrix = observed_matrix.set_index(['userid'],drop=True) 

C:\Users\Tim\AppData\Local\Temp/ipykernel_8080/58843463.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  observed_matrix = users.drop('movies', 1).join(
C:\Users\Tim\AppData\Local\Temp/ipykernel_8080/58843463.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  pd.get_dummies(pd.DataFrame(


In [19]:
observed_matrix

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,1673.0,1674.0,1675.0,1676.0,1677.0,1678.0,1679.0,1680.0,1681.0,1682.0
userid,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
940,0,0,0,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
941,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
P_observed = observed_matrix.values.sum()

100000

In [22]:
inversePropensities = observed_matrix.copy()
for col in inversePropensities.columns:
    inversePropensities[col].values[:] = 0

In [11]:
inversePropensities = np.ones(
    (num_users, num_items), dtype=np.longdouble) * scale / num_observation
inversePropensities

array([[7.93063, 7.93063, 7.93063, ..., 7.93063, 7.93063, 7.93063],
       [7.93063, 7.93063, 7.93063, ..., 7.93063, 7.93063, 7.93063],
       [7.93063, 7.93063, 7.93063, ..., 7.93063, 7.93063, 7.93063],
       ...,
       [7.93063, 7.93063, 7.93063, ..., 7.93063, 7.93063, 7.93063],
       [7.93063, 7.93063, 7.93063, ..., 7.93063, 7.93063, 7.93063],
       [7.93063, 7.93063, 7.93063, ..., 7.93063, 7.93063, 7.93063]])